In [ ]:
import h3 
import xarray as xr
import rasterio
from shapely.geometry import Polygon
import geopandas as gpd

Se leen los datos de velocidad del viento que contienen toda la extension del proyecto. Se busca utilizar la extensión del mismo para definir la región a llenar con hexágonos.

In [4]:
# wind_speed
path = '/home/jta/Documentos/raster_variables/ahpAmbiental/COL_wind-speed_10m.tif'
ds_wind = rasterio.open(path)

Para generar una malla de hexagonos se genera un poligono que contiene los limites del raster.

In [5]:
ext = ds_wind.bounds
up_l_corner = (ext[0],ext[-1])
up_r_corner = (ext[2],ext[-1])
down_l_corner = (ext[0],ext[1])
down_r_cornor = (ext[2],ext[1])
# Creacion poligono
polygon = Polygon(
    (
        down_l_corner,down_r_cornor,
        up_r_corner,up_l_corner
    )
)


la funcion `geo_to_cells` genera la malla de hexagonos, a esta se le debe suministrar la resolucion con el tamano de los hexagonos. Esta regresa una lista de `str` que contienen el codigo de cada hexagonos

In [6]:
# Creacion de una malla de hexagonos
resolution = 8
hexagons = h3.geo_to_cells(polygon, resolution)


De cada hexagono se pueden extraer los vertices de cada hexagono, esto se puede convertir en un geodataframe para almacenar la informacion asociada a cada variable. Donde cada columna del geodataframe corresponde a un atributo especifico

In [9]:
# Genera lista con la extension de 
geometry = [Polygon(h3.cell_to_boundary(h)) for h in hexagons]

gdf_hex = gpd.GeoDataFrame(
    {'h3': hexagons},
    geometry=geometry,
    crs="EPSG:4326"
)
# cambio de epsg
epsg_on = '9377'
gdf_hex.to_crs(epsg_on)

,h3,geometry
0,88668e9169fffff,"POLYGON ((5976161.357 -8225907.017, 5975689.45..."
1,888a6b49bdfffff,"POLYGON ((6995828.723 -7728090.338, 6995352.09..."
2,886750560bfffff,"POLYGON ((7143147.685 -8228422.4, 7142691.086 ..."
3,8867ae0455fffff,"POLYGON ((6234512.052 -8371854.208, 6234052.45..."
4,886771b2d3fffff,"POLYGON ((7466765.241 -8271250.169, 7466318.01..."
...,...,...
5931349,88670d4467fffff,"POLYGON ((7058883.413 -8324665.558, 7058431.60..."
5931350,8866159651fffff,"POLYGON ((6521052.843 -8239299.279, 6520586.59..."
5931351,886694a163fffff,"POLYGON ((5601402.959 -8329797.051, 5600934.71..."
5931352,8867adac41fffff,"POLYGON ((6480266.406 -8351998.314, 6479807.83..."


In [8]:
# Lectura de datos
path = '/home/jta/Documentos/raster_variables/datos_cris/especies_densidades/especies_migratorias_final'
gdf_migratorias = gpd.read_file(path)

In [ ]:
# Funcion para extraer la densidad
def densidad_especies(geometry):
    pass